In [24]:
import keras
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import Input,ZeroPadding2D,InputLayer
from keras.models import Model

import warnings
warnings.filterwarnings('ignore')

In [25]:
train_data=pd.read_csv("Train.csv")
target=train_data['label']
train_data.drop('label', inplace=True, axis=1)
np.shape(train_data)

(11995, 784)

In [26]:
np.shape(target)

(11995,)

In [27]:
test_data=pd.read_csv("Test.csv")
np.shape(test_data)

(3000, 784)

In [28]:
train_data=np.array(train_data)
test_data=np.array(test_data)
target=np.array(target)

In [29]:
train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

In [30]:
np.shape(train_data)

(11995, 28, 28, 1)

In [31]:
#Pre-processing

train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_data /= 255
test_data /= 255
print('x_train shape:', train_data.shape)
print(train_data.shape[0], 'train samples')
print(test_data.shape[0], 'test samples')

x_train shape: (11995, 28, 28, 1)
11995 train samples
3000 test samples


In [32]:
#One-Hot encoding of target
target = keras.utils.to_categorical(target, 10)
np.shape(target)

(11995, 10)

In [33]:
#Model building

model=Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", 
          input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))

model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="valid", activation='relu'))

model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(LeakyReLU())
model.add(Dense(128))
model.add(LeakyReLU())

model.add(Dense(10, activation='softmax'))

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 256)         0         
__________

In [35]:
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [36]:
batch_size=128

In [37]:
from keras.preprocessing.image import ImageDataGenerator

In [38]:
#Data Augmentation
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

In [39]:
train_generator = gen.flow(train_data, target, batch_size=batch_size)

In [40]:
len(train_data)

11995

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
train_x,test_x,train_y,test_y=train_test_split(train_data,target,test_size=0.2)

In [43]:
model.fit_generator(train_generator, steps_per_epoch=len(train_data)//batch_size, 
                    epochs=20,validation_data=(test_x,test_y))

Epoch 1/20
93/93 [==============================] - 185s 2s/step - loss: 1.1280 - acc: 0.5681 - val_loss: 0.6395 - val_acc: 0.7532
Epoch 2/20
93/93 [==============================] - 182s 2s/step - loss: 0.6805 - acc: 0.7379 - val_loss: 0.5464 - val_acc: 0.7662
Epoch 3/20
93/93 [==============================] - 183s 2s/step - loss: 0.6257 - acc: 0.7519 - val_loss: 0.4986 - val_acc: 0.7907
Epoch 4/20
93/93 [==============================] - 194s 2s/step - loss: 0.5741 - acc: 0.7752 - val_loss: 0.4706 - val_acc: 0.8178
Epoch 5/20
93/93 [==============================] - 187s 2s/step - loss: 0.5378 - acc: 0.7890 - val_loss: 0.4124 - val_acc: 0.8378
Epoch 6/20
93/93 [==============================] - 196s 2s/step - loss: 0.5230 - acc: 0.7949 - val_loss: 0.3912 - val_acc: 0.8391
Epoch 7/20
93/93 [==============================] - 196s 2s/step - loss: 0.4957 - acc: 0.8102 - val_loss: 0.3700 - val_acc: 0.8533
Epoch 8/20
93/93 [==============================] - 201s 2s/step - loss: 0.4857 - a

In [44]:
#Predictions on test data
pred=model.predict(x=test_data)
pred=np.argmax(a=pred,axis=1)
np.shape(pred)
pred=pd.DataFrame(pred)
pred.to_csv("predictions.csv")